<a href="https://colab.research.google.com/github/msamwelmollel/swahili-prompt-translation-for-swahili/blob/main/Uploading_prompts_to_be_translated_to_an_Argilla_Space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>